# Parameter Space Explorer Dashboard

Interactive exploration of the 36-configuration simulation grid.
Select **Cap**, **ATR**, and **Deductible** to view:
- **Wealth Fan Chart** - Percentile envelopes of asset trajectories (insured vs uninsured)
- **Growth Rate Distribution** - KDE of time-average growth rates
- **Ruin Probability Curve** - Cumulative ruin probability over 50 years
- **Key Metrics** - Summary comparison table

**Note:** Wealth trajectories are approximated via exponential interpolation from per-simulation
growth rates: `assets(t) = A_0 * exp(g_i * t)`. This is exact at t=0 and t=T, and produces
correct percentile envelopes since exp is monotone.

**Prerequisites:** Run `1. process_vol_sim_results.ipynb` first to build `cache/dashboard_cache.pkl`.

In [ ]:
import pickle
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML
from pathlib import Path

In [ ]:
# Load dashboard cache
cache_path = Path("cache/dashboard_cache.pkl")
with open(cache_path, "rb") as f:
    cache = pickle.load(f)

configs = cache["configs"]
crn_pairs = cache["crn_pairs"]
param_values = cache["param_values"]
pctile_levels = cache["percentile_levels"]
q_levels = cache["quantile_levels"]

print(f"Loaded {len(configs)} configs, {len(crn_pairs)} CRN pairs")
print(f"Cap: {param_values['Cap']}")
print(f"ATR: {param_values['ATR']}")
print(f"Ded: {param_values['Ded']}")

In [ ]:
def find_config_key(cap, atr, ded):
    """Find the config key matching the given insured parameters."""
    for key, cfg in configs.items():
        if (cfg.get("Cap") == cap
            and cfg.get("ATR") == atr
            and cfg.get("Ded") == ded
            and not cfg.get("NOINS")):
            return key
    return None


def find_noins_key(cap, atr):
    """Find the NOINS config key matching the given Cap and ATR."""
    for key, cfg in configs.items():
        if (cfg.get("Cap") == cap
            and cfg.get("ATR") == atr
            and cfg.get("NOINS")):
            return key
    return None


def fmt_amount(n):
    """Format a dollar amount."""
    if n >= 1e9: return f"${n/1e9:.0f}B"
    if n >= 1e6: return f"${n/1e6:.0f}M"
    if n >= 1e3: return f"${n/1e3:.0f}K"
    return f"${n:.0f}"


def get_ruin_val(ruin_dict, year):
    """Get ruin probability for a given year, handling int/str keys."""
    for k, v in ruin_dict.items():
        if int(k) == year:
            return v
    return None

In [ ]:
# Create widgets
cap_dropdown = widgets.Dropdown(
    options=[(fmt_amount(v), v) for v in param_values["Cap"]],
    value=param_values["Cap"][0],
    description="Cap:",
    style={"description_width": "auto"},
)
atr_dropdown = widgets.Dropdown(
    options=[(str(v), v) for v in param_values["ATR"]],
    value=param_values["ATR"][1] if len(param_values["ATR"]) > 1 else param_values["ATR"][0],
    description="ATR:",
    style={"description_width": "auto"},
)
ded_dropdown = widgets.Dropdown(
    options=[(fmt_amount(v), v) for v in param_values["Ded"]],
    value=param_values["Ded"][1] if len(param_values["Ded"]) > 1 else param_values["Ded"][0],
    description="Deductible:",
    style={"description_width": "auto"},
)

output_area = widgets.Output()

In [ ]:
# Color scheme
INS_COLOR = "rgb(31,119,180)"   # Blue for insured
NI_COLOR = "rgb(255,127,14)"    # Orange for uninsured
INS_FILLS = ["rgba(31,119,180,0.08)", "rgba(31,119,180,0.15)", "rgba(31,119,180,0.30)"]
NI_FILLS = ["rgba(255,127,14,0.08)", "rgba(255,127,14,0.15)", "rgba(255,127,14,0.30)"]
FAN_BANDS = [(5, 95), (25, 75), (40, 60)]  # Outer to inner


def build_fan_traces(cfg, color, fills, name_prefix, show_legend=True):
    """Build wealth fan chart traces from growth rate quantiles."""
    traces = []
    n_years = cfg["n_years"]
    initial = cfg["initial_assets"]
    years = np.arange(n_years + 1)
    gr_qs = cfg["growth_rate_qs"]

    for (lo, hi), fill_color in zip(FAN_BANDS, fills):
        lo_idx = int(lo / 100 * (len(gr_qs) - 1))
        hi_idx = int(hi / 100 * (len(gr_qs) - 1))
        g_lo = gr_qs[lo_idx]
        g_hi = gr_qs[hi_idx]

        y_lo = initial * np.exp(g_lo * years)
        y_hi = initial * np.exp(g_hi * years)

        traces.append(go.Scatter(
            x=np.concatenate([years, years[::-1]]),
            y=np.concatenate([y_hi, y_lo[::-1]]),
            fill="toself", fillcolor=fill_color,
            line=dict(width=0),
            name=f"{name_prefix} P{lo}-P{hi}",
            showlegend=show_legend,
            legendgroup=name_prefix,
        ))

    # Median line
    g_med = gr_qs[len(gr_qs) // 2]
    y_med = initial * np.exp(g_med * years)
    traces.append(go.Scatter(
        x=years, y=y_med,
        line=dict(color=color, width=2.5),
        name=f"{name_prefix} Median",
        legendgroup=name_prefix,
    ))

    return traces


def build_growth_rate_traces(cfg, color, name, dash=None):
    """Build growth rate distribution trace from quantiles (approximate PDF)."""
    gr_qs = cfg["growth_rate_qs"]
    dq = np.diff(q_levels)
    dg = np.diff(gr_qs)
    dg[dg == 0] = 1e-12
    pdf = dq / dg
    midpoints = (gr_qs[:-1] + gr_qs[1:]) / 2

    # Clip extreme tails for cleaner visualization
    mask = (q_levels[:-1] >= 0.01) & (q_levels[1:] <= 0.99)
    line_style = dict(color=color, width=1.5)
    if dash:
        line_style["dash"] = dash

    return go.Scatter(
        x=midpoints[mask], y=pdf[mask],
        fill="tozeroy",
        fillcolor=color.replace("rgb", "rgba").replace(")", ",0.2)"),
        line=line_style,
        name=name,
    )


def build_ruin_traces(cfg, color, name, dash=None):
    """Build ruin probability curve trace."""
    ruin = cfg["ruin_probability"]
    years_sorted = sorted(ruin.keys(), key=int)
    xs = [int(y) for y in years_sorted]
    ys = [ruin[y] for y in years_sorted]

    line_style = dict(color=color, width=2.5)
    if dash:
        line_style["dash"] = dash

    return go.Scatter(
        x=xs, y=ys,
        mode="lines+markers",
        line=line_style,
        marker=dict(size=5),
        name=name,
    )

In [ ]:
def update_dashboard(change=None):
    """Rebuild the full dashboard on parameter change."""
    cap = cap_dropdown.value
    atr = atr_dropdown.value
    ded = ded_dropdown.value

    ins_key = find_config_key(cap, atr, ded)
    noins_key = find_noins_key(cap, atr)

    if ins_key is None:
        with output_area:
            output_area.clear_output(wait=True)
            print(f"No insured data found for Cap={fmt_amount(cap)}, ATR={atr}, Ded={fmt_amount(ded)}")
            # Show available combos
            avail = [(v['Cap'], v['ATR'], v['Ded']) for v in configs.values() if not v.get('NOINS')]
            print(f"Available: {sorted(set(avail))}")
        return

    ins = configs[ins_key]
    noins = configs.get(noins_key) if noins_key else None

    # ---- Build 2x2 subplot figure ----
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            "Wealth Fan Chart (Log Scale)",
            "Growth Rate Distribution",
            "Cumulative Ruin Probability",
            "Retained Loss Fan Chart",
        ],
        vertical_spacing=0.12,
        horizontal_spacing=0.10,
    )

    # ===== Panel 1: Wealth Fan Chart =====
    for tr in build_fan_traces(ins, INS_COLOR, INS_FILLS, "Insured"):
        fig.add_trace(tr, row=1, col=1)

    if noins:
        for tr in build_fan_traces(noins, NI_COLOR, NI_FILLS, "Uninsured"):
            fig.add_trace(tr, row=1, col=1)

    # Ruin threshold
    insolvency_tol = 10_000
    fig.add_hline(
        y=insolvency_tol, line_dash="dot", line_color="red",
        annotation_text="Ruin threshold", annotation_position="bottom left",
        row=1, col=1,
    )

    fig.update_yaxes(type="log", title_text="Assets ($)", row=1, col=1)
    fig.update_xaxes(title_text="Year", row=1, col=1)

    # ===== Panel 2: Growth Rate Distribution =====
    fig.add_trace(
        build_growth_rate_traces(ins, INS_COLOR, "Insured"),
        row=1, col=2,
    )
    if noins:
        fig.add_trace(
            build_growth_rate_traces(noins, NI_COLOR, "Uninsured", dash="dash"),
            row=1, col=2,
        )

    # Mean vertical lines
    fig.add_vline(
        x=ins["growth_rate_mean"], line_dash="solid", line_color=INS_COLOR,
        annotation_text=f"Ins mean: {ins['growth_rate_mean']:.4f}",
        row=1, col=2,
    )
    if noins:
        fig.add_vline(
            x=noins["growth_rate_mean"], line_dash="dash", line_color=NI_COLOR,
            annotation_text=f"Unins mean: {noins['growth_rate_mean']:.4f}",
            row=1, col=2,
        )

    fig.update_xaxes(title_text="Annualized Growth Rate", row=1, col=2)
    fig.update_yaxes(title_text="Density", row=1, col=2)

    # ===== Panel 3: Ruin Probability =====
    fig.add_trace(
        build_ruin_traces(ins, INS_COLOR, "Insured"),
        row=2, col=1,
    )
    if noins:
        fig.add_trace(
            build_ruin_traces(noins, NI_COLOR, "Uninsured", dash="dash"),
            row=2, col=1,
        )

    fig.update_xaxes(title_text="Year", row=2, col=1)
    fig.update_yaxes(title_text="Cumulative Ruin Probability", tickformat=".2%", row=2, col=1)

    # ===== Panel 4: Retained Loss Fan Chart =====
    n_years = ins["n_years"]
    years = np.arange(1, n_years + 1)
    pctile_idx = {p: i for i, p in enumerate(pctile_levels)}

    ts_ret = ins["ts_retained_losses_pctiles"]
    band_pairs = [(5, 95), (25, 75)]
    band_fills_ins = ["rgba(31,119,180,0.12)", "rgba(31,119,180,0.25)"]

    for (lo, hi), fill_c in zip(band_pairs, band_fills_ins):
        y_lo = ts_ret[:, pctile_idx[lo]]
        y_hi = ts_ret[:, pctile_idx[hi]]
        fig.add_trace(go.Scatter(
            x=np.concatenate([years, years[::-1]]),
            y=np.concatenate([y_hi, y_lo[::-1]]),
            fill="toself", fillcolor=fill_c,
            line=dict(width=0), showlegend=False,
        ), row=2, col=2)

    fig.add_trace(go.Scatter(
        x=years, y=ts_ret[:, pctile_idx[50]],
        line=dict(color=INS_COLOR, width=2),
        name="Insured Median Retained",
    ), row=2, col=2)

    if noins:
        ts_ret_ni = noins["ts_retained_losses_pctiles"]
        for (lo, hi), fill_c in zip(band_pairs, ["rgba(255,127,14,0.12)", "rgba(255,127,14,0.25)"]):
            y_lo = ts_ret_ni[:, pctile_idx[lo]]
            y_hi = ts_ret_ni[:, pctile_idx[hi]]
            fig.add_trace(go.Scatter(
                x=np.concatenate([years, years[::-1]]),
                y=np.concatenate([y_hi, y_lo[::-1]]),
                fill="toself", fillcolor=fill_c,
                line=dict(width=0), showlegend=False,
            ), row=2, col=2)

        fig.add_trace(go.Scatter(
            x=years, y=ts_ret_ni[:, pctile_idx[50]],
            line=dict(color=NI_COLOR, width=2, dash="dash"),
            name="Uninsured Median Retained",
        ), row=2, col=2)

    fig.update_xaxes(title_text="Year", row=2, col=2)
    fig.update_yaxes(title_text="Annual Retained Loss ($)", row=2, col=2)

    # ===== Layout =====
    title = f"Cap={fmt_amount(cap)} | ATR={atr} | Ded={fmt_amount(ded)}"
    fig.update_layout(
        height=850, width=1300,
        title_text=f"Parameter Space Explorer: {title}",
        title_font_size=16,
        showlegend=True,
        legend=dict(font=dict(size=9), orientation="h", yanchor="bottom", y=-0.15),
        margin=dict(t=80, b=100),
    )

    # ===== Metrics table (HTML below the chart) =====
    def pct(v):
        return f"{v:.2%}" if v is not None else "N/A"

    rows_html = []
    metrics = [
        ("Mean Growth Rate", f"{ins['growth_rate_mean']:.5f}",
         f"{noins['growth_rate_mean']:.5f}" if noins else "-"),
        ("Median Growth Rate", f"{ins['growth_rate_median']:.5f}",
         f"{noins['growth_rate_median']:.5f}" if noins else "-"),
        ("Std Dev Growth Rate", f"{ins['growth_rate_std']:.5f}",
         f"{noins['growth_rate_std']:.5f}" if noins else "-"),
        ("Mean Final Assets", fmt_amount(ins['final_assets_mean']),
         fmt_amount(noins['final_assets_mean']) if noins else "-"),
        ("Median Final Assets", fmt_amount(ins['final_assets_median']),
         fmt_amount(noins['final_assets_median']) if noins else "-"),
        ("Ruin Prob (Yr 10)", pct(get_ruin_val(ins['ruin_probability'], 10)),
         pct(get_ruin_val(noins['ruin_probability'], 10)) if noins else "-"),
        ("Ruin Prob (Yr 25)", pct(get_ruin_val(ins['ruin_probability'], 25)),
         pct(get_ruin_val(noins['ruin_probability'], 25)) if noins else "-"),
        ("Ruin Prob (Yr 50)", pct(get_ruin_val(ins['ruin_probability'], 50)),
         pct(get_ruin_val(noins['ruin_probability'], 50)) if noins else "-"),
    ]
    if noins:
        lift = ins['growth_rate_mean'] - noins['growth_rate_mean']
        lift_bps = lift * 10_000
        metrics.append(("Growth Lift", f"{lift:.5f} ({lift_bps:.1f} bps)", "-"))

    for label, v_ins, v_ni in metrics:
        rows_html.append(f"<tr><td style='padding:4px 12px'><b>{label}</b></td>"
                         f"<td style='padding:4px 12px; text-align:right'>{v_ins}</td>"
                         f"<td style='padding:4px 12px; text-align:right'>{v_ni}</td></tr>")

    table_html = (
        "<table style='border-collapse:collapse; font-family:monospace; font-size:13px; margin-top:12px'>"
        "<tr style='border-bottom:2px solid #333'>"
        "<th style='padding:4px 12px; text-align:left'>Metric</th>"
        "<th style='padding:4px 12px; text-align:right; color:#1f77b4'>Insured</th>"
        "<th style='padding:4px 12px; text-align:right; color:#ff7f0e'>Uninsured</th></tr>"
        + "\n".join(rows_html)
        + "</table>"
    )

    with output_area:
        output_area.clear_output(wait=True)
        fig.show()
        display(HTML(table_html))

In [ ]:
# Wire up callbacks and display
cap_dropdown.observe(update_dashboard, names="value")
atr_dropdown.observe(update_dashboard, names="value")
ded_dropdown.observe(update_dashboard, names="value")

controls = widgets.HBox(
    [cap_dropdown, atr_dropdown, ded_dropdown],
    layout=widgets.Layout(gap="20px"),
)
display(controls)
display(output_area)

# Initial render
update_dashboard()

## Quick Comparison: All Deductibles for Current Cap/ATR

Side-by-side view of key metrics across all deductible levels for the currently selected Cap and ATR.

In [ ]:
def show_deductible_comparison():
    """Show all deductible levels side-by-side for the current Cap and ATR."""
    cap = cap_dropdown.value
    atr = atr_dropdown.value
    noins_key = find_noins_key(cap, atr)
    noins = configs.get(noins_key) if noins_key else None

    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=[
            "Mean Growth Rate by Deductible",
            "Ruin Probability (Year 50)",
            "Median Final Assets",
        ],
        horizontal_spacing=0.08,
    )

    deds = []
    gr_means = []
    ruin_50s = []
    med_fas = []

    for ded in param_values["Ded"]:
        k = find_config_key(cap, atr, ded)
        if k is None:
            continue
        c = configs[k]
        deds.append(fmt_amount(ded))
        gr_means.append(c["growth_rate_mean"])
        ruin_50s.append(get_ruin_val(c["ruin_probability"], 50) or 0)
        med_fas.append(c["final_assets_median"])

    if not deds:
        print(f"No data for Cap={fmt_amount(cap)}, ATR={atr}")
        return

    # Add uninsured baseline
    if noins:
        deds.append("No Ins")
        gr_means.append(noins["growth_rate_mean"])
        ruin_50s.append(get_ruin_val(noins["ruin_probability"], 50) or 0)
        med_fas.append(noins["final_assets_median"])

    colors = [INS_COLOR] * (len(deds) - (1 if noins else 0)) + ([NI_COLOR] if noins else [])

    fig.add_trace(go.Bar(x=deds, y=gr_means, marker_color=colors, showlegend=False), row=1, col=1)
    fig.add_trace(go.Bar(x=deds, y=ruin_50s, marker_color=colors, showlegend=False), row=1, col=2)
    fig.add_trace(go.Bar(x=deds, y=med_fas, marker_color=colors, showlegend=False), row=1, col=3)

    fig.update_yaxes(tickformat=".4f", title_text="Growth Rate", row=1, col=1)
    fig.update_yaxes(tickformat=".2%", title_text="Ruin Prob", row=1, col=2)
    fig.update_yaxes(title_text="Median Assets ($)", row=1, col=3)

    fig.update_layout(
        height=400, width=1300,
        title_text=f"Deductible Comparison: Cap={fmt_amount(cap)}, ATR={atr}",
        title_font_size=14,
    )
    fig.show()

    # Identify optimal deductible
    ins_deds = [(d, g) for d, g in zip(deds, gr_means) if d != "No Ins"]
    if ins_deds:
        best_ded, best_gr = max(ins_deds, key=lambda x: x[1])
        worst_ded, worst_gr = min(ins_deds, key=lambda x: x[1])
        penalty_bps = (best_gr - worst_gr) * 10_000
        print(f"Optimal deductible: {best_ded} (growth rate = {best_gr:.5f})")
        print(f"Worst deductible: {worst_ded} (growth rate = {worst_gr:.5f})")
        print(f"Cost of getting it wrong: {penalty_bps:.1f} basis points")


show_deductible_comparison()